In [1]:
#To increase cell width of ipynb
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import tensorflow as tf
from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random

/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [2]:
def extract_data(self):
    train = []
    test = []
    train_labels = []
    test_labels = []
    label_names = []
    IMG_SIZE = 128
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
    train_features = np.empty((0,1280), float)
    test_features = np.empty((0,1280), float)
    
    mobilenet = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, weights="imagenet")
    mobilenet.trainable = False
    mobilenet = tf.keras.Model(inputs=mobilenet.input, outputs=[mobilenet.layers[-2].output])
#     mobilenet.summary()
#     print(mobilenet.layers[-1].output)
    
    print("\n..........loading dataset from disk..........\n")

    for train_or_test in os.listdir(data_path):
        
        projections_path = os.path.join(data_path, train_or_test)

        projections = os.listdir(projections_path)
        
        labels_path = os.path.join(projections_path, projections[0])

        labels =  os.listdir(labels_path)

        for label in labels:

            images_names =  os.listdir(os.path.join(labels_path, label))

            for i in images_names:
                image1 = cv.imread(data_path+"/"+str(train_or_test)+"/"+str(projections[0])+"/"+str(label)+"/"+str(i), cv.IMREAD_GRAYSCALE)
                image2 = cv.imread(data_path+"/"+str(train_or_test)+"/"+str(projections[1])+"/"+str(label)+"/"+str(i), cv.IMREAD_GRAYSCALE)
                image3 = cv.imread(data_path+"/"+str(train_or_test)+"/"+str(projections[2])+"/"+str(label)+"/"+str(i), cv.IMREAD_GRAYSCALE)
                image = cv.merge([image1, image2, image3])
                if str(train_or_test) == "train":
                    train_labels.append(label)
                    train = np.array([image], dtype="float") / 255.0
                    train_features = np.concatenate((train_features, mobilenet.predict(train)), axis=0)
                    
                elif str(train_or_test) == "test":
                    test_labels.append(label)
                    test = np.array([image], dtype="float") / 255.0
                    test_features = np.concatenate((test_features, mobilenet.predict(test)), axis=0)

            
    print(train_features.shape)
    print(test_features.shape)             
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)
    label_names = np.unique(train_labels)
    print(label_names.shape)

#     return train_features_flatten, train_labels, test_features_flatten, test_labels, label_names
    return train_features, train_labels, test_features, test_labels, label_names

In [3]:
if __name__ == "__main__":
    
    data_path = '../../RGB-D/image_dataset'
    
    np_train_path = '../../RGB-D/extracted_features/MobileNetV2/train.npy.gz'
    np_test_path = '../../RGB-D/extracted_features/MobileNetV2/test.npy.gz'
    np_train_label_path = '../../RGB-D/extracted_features/MobileNetV2/train_labels.npy'
    np_test_label_path = '../../RGB-D/extracted_features/MobileNetV2/test_labels.npy'
    np_label_names_path = '../../RGB-D/extracted_features/MobileNetV2/label_names.npy'

    train_data, train_labels, test_data, test_labels, label_names = extract_data(data_path)

    os.makedirs(os.path.dirname(np_train_path), exist_ok=True)

    with gzip.GzipFile(np_train_path, "w") as f:
        np.save(f, train_data)
    with gzip.GzipFile(np_test_path, "w") as f:
        np.save(f, test_data)

    np.save(np_train_label_path, train_labels)
    np.save(np_test_label_path, test_labels)
    np.save(np_label_names_path, label_names)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

..........loading dataset from disk..........

(38262, 1280)
(9503, 1280)
(51,)
